## Cargar el modelo 

In [ ]:
from Libraries_functions import *

#df_data,df_route = load_df('D:\Caracuel\WaveNet-seaBackGroundNoise\Datasets\Data\dataframe.csv')
#df_data,df_route = load_df('D:\Caracuel\WaveNet-seaBackGroundNoise\Code\generatingData\dataframe.csv')

df_data,df_route = load_df('./dataframe.csv')


####
file_len = len(df_data.data[0])
files_num = df_data.shape[0]
col_num = df_data.shape[1]

print('Main DF info:\n')
print('Records length:', file_len)
#print('Records range: [-1, 1]')
print('Nº of records:', files_num)
print('Nº of columns:', col_num, '. With pairs 1-2, 1-3 or 1-4.')

df_data.head()


###
df_lluvia=pd.DataFrame()
lluvia_mezclada=[]
lluvia=[]
for i in tqdm(range(len(df_data))):
    if(i>=df_data.shape[0]/4-1 and i<df_data.shape[0]/4+499-1):
        lluvia.append(df_data.data[i])
        lluvia_mezclada.append(df_data.noise_original[i])

df_lluvia['data']=lluvia
df_lluvia['noise_original']=lluvia


####
option = 3

####
if option == 3:
    sistema = platform.system()
    if (sistema =="Linux"):
        strategy = tf.distribute.MirroredStrategy()
        print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

        # Open a strategy scope.
        with strategy.scope():
        # Everything that creates variables should be under the strategy scope.
        # In general this is only model construction & `compile()`.
            # Call to the original u-Net
            model = unet(file_len)
            adam=tf.keras.optimizers.Adam(learning_rate=0.01)
            model.compile(optimizer=adam, loss='mse', metrics=['accuracy'])
    else:
        model = unet(file_len)
        adam = tf.keras.optimizers.Adam(learning_rate=0.01)
        model.compile(optimizer=adam, loss='mse', metrics=['accuracy'])
    model.summary()
####
X, y = chunk4unet(df_lluvia, 'data', 'noise_original')

X = np.reshape(X, (np.shape(X)[0], np.shape(X)[1], 1))
y = np.reshape(y, (np.shape(y)[0], np.shape(y)[1], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


###
model.load_weights("./weights_001.hdf5")
pred_outp=model.predict(X_test)
print(np.shape(pred_outp))

audio_output = np.reshape(pred_outp, 1,(np.shape(pred_outp)))

export_wav(pred_outp)